# HNN File Generator

In [1]:
import fileio
import paramrw
import params_default
import csd_functions
import numpy as np

In [2]:
gid_dict, p = paramrw.read('example_param.param') # Base file to make edits on

### Manual paramater edits

In [3]:
p['t_evprox_2'] = 0

['prng_seedcore_input_prox': 13
['prng_seedcore_input_dist': 14
['prng_seedcore_extpois': 4
['prng_seedcore_extgauss': 4
['prng_seedcore_evprox_1': 4
['prng_seedcore_evdist_1': 4
['prng_seedcore_evprox_2':]= 4
['prng_seedcore_evdist_2']: 0

### Define parameters to sweep over

In [4]:
file_prefix = 'test_sweep-' #prefix appended to each file name
file_dir = 'D:/Jones_Lab/hnn_params/test_sweep/' #directory to store param files

param_names = ['t_evprox_1','t_evdist_1'] #List dictionary keys to edit
param_val = [np.linspace(10,150,3), np.linspace(10,150,3)]

num_params = len(param_names)

#Values stored under v1,v2; param suffix stored under s1,s2
param_sets = [ [[s1,s2], [v1,v2]]  for s1,v1 in enumerate(param_val[0]) for s2,v2 in enumerate(param_val[1])]


dict_set = [] #Store new dictionaries
dict_names = [] #Stor file names

for p_set in param_sets:
    temp_dict = p.copy() #Copy template parameters

    p_suffix = p_set[0]
    p_val = p_set[1]

    f_name = file_prefix
    for idx in range(num_params):
        if idx < num_params-1:
            f_name = f_name + param_names[idx] + '_' + str(p_suffix[idx]) + '_' #Update file name with edited param names
        else:
            f_name = f_name + param_names[idx] + '_' + str(p_suffix[idx])

        temp_dict[param_names[idx]] = p_val[idx]


    #Edit dictionary names
    temp_dict['sim_prefix'] = f_name
    temp_dict['expmt_groups'] = '{' + f_name + '}'
    
    dict_set.append(temp_dict)
    dict_names.append(f_name)

    f_string= file_dir + f_name + '.param'
    #Create files
    paramrw.write(f_string,temp_dict,gid_dict)


In [5]:
paramrw.write('test_param_write.param', p, gid_dict)